In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import random

# Settings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epoch = 20
batch_size = 100
learning_rate = 0.001

transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32)
])

In [8]:
whole_image = torch.load('ft_data_image')
augmented_images_label = torch.load('augmented_images_label.pt')
whole_label = augmented_images_label + augmented_images_label + augmented_images_label + augmented_images_label
whole_label = torch.stack(whole_label)
#whole_image = whole_image[:20000]
#whole_label = whole_label[:20000]

torch.manual_seed(77777)
pmtn = torch.randperm(whole_image.size()[0])          #seed를 고정해도 torch.randperm이 선언될때마다 각각은 고정되지만
whole_image=whole_image[pmtn]                         #서로 다른 pmtn을 갖는다 그래서 하나로 고정
whole_label=whole_label[pmtn]
whole_image = whole_image[:100]
whole_label = whole_label[:100]
print(whole_image.size())
print(whole_label.size())

torch.Size([100, 3, 32, 32])
torch.Size([100])


In [9]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from model_resnet import ResNet
from model_resnet import ResidualBlock

num_epoch = 1
learning_rate = 0.001

# Model Load
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
model = torch.load('resnet_trained_model.ckpt')
model.load_state_dict(torch.load('resnet_trained_model_state.ckpt'))

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adadelta(model.parameters(), lr=0.00001, rho=0.9, eps=1e-06, weight_decay=0)

# For updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [10]:
#Test the pre_trained Model

# 1. Loading CIFAR dataset
test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False,
                                            transform=transforms.ToTensor(),
                                            download=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=False)

model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the pre-trained model on the test images: {} %'.format(100 * correct / total))

Files already downloaded and verified
Accuracy of the pre-trained model on the test images: 88.49 %


In [11]:
total_step = whole_image.size()[0] // batch_size
curr_lr = learning_rate
model.train()

for epoch in range(num_epoch):
    correct = 0
    total = 0
    
    # Transform & Shuffle
    whole_image = transform(whole_image.to(device))
    whole_label = whole_label.to(device)
    
    torch.manual_seed(epoch)
    pmtn = torch.randperm(whole_image.size()[0])          #seed를 고정해도 torch.randperm이 선언될때마다 각각은 고정되지만
    whole_image=whole_image[pmtn]                         #서로 다른 pmtn을 갖는다 그래서 하나로 고정
    whole_label=whole_label[pmtn]                 

    for i in range(whole_image.size()[0] // batch_size):
        images = whole_image[i*batch_size:(i+1)*batch_size]
        labels = whole_label[i*batch_size:(i+1)*batch_size]
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

        acc = 100 * correct / total
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} | Accuracy: {:.4f}"
                   .format(epoch+1, num_epoch, i+1, total_step, loss.item(), acc))
        
     # Decay learning rate
    if ((epoch+1)%2==0):
        curr_lr /= 3
        update_lr(optimizer, curr_lr)
        

In [12]:
import torch
import torchvision
import torchvision.transforms as transforms

# Device configuration
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(device)

# 1. Loading CIFAR dataset
#test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
#                                            train=False,
#                                            transform=transforms.ToTensor(),
#                                            download=True)

#test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
#                                          batch_size=100,
#                                          shuffle=False)

"""for images, labels in test_loader:
    print(images.shape)   # torch.Size([100, 3, 32, 32])
    print(labels.shape)   # torch.Size([100])
    print(type(images))"""

# Test the model
#model = torch.load('resnet_fine_tuned_model_epoch5.ckpt')
#model.load_state_dict(torch.load('resnet_fine_tuned_model_state_epoch5.ckpt'))
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the fine-tuned model on the test images: {} %'.format(100 * correct / total))


Accuracy of the fine-tuned model on the test images: 88.59 %


In [17]:
torch.save(model,'resnet_fine_tuned_modelA.ckpt') #전체모델저장
torch.save(model.state_dict(),'resnet_fine_tuned_modelA_state.ckpt')
torch.save({
        'model':model.state_dict(),
        'optimizer':optimizer.state_dict()},
        'resnet_fine_tuned_modelA_all.tar')